In [1]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

lines = []
i = 0
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if(i > 0): #this part is added to insure the 1st line in the csvfile which include the headers is not included
             lines.append(line)
        i = i + 1
        
images = []
measurments = []
for line in lines:
    #print(line)
    # get the image path from the file
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'IMG/'+ filename
    #print('current_path', current_path)
    image = cv2.imread(current_path)
   # plt.imshow(image)
    images.append(image)
    # get the steering measurmement (labels) from the file
   # measurment = float(line[3])
    measurment = line[3]
    measurments.append(measurment)

# convert features and labels to numpy array as that's the format keras requires  
print("Sample of labels", measurments[4823] )
X_train = np.array(images)
y_train = np.array(measurments) 
print(X_train[0].shape, ' shape of train samples')
print("size of training set", len(X_train))



# build a very simple model to test on it
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense  

model = Sequential()
model.add(Flatten(input_shape=(160,320,3))) 
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)
#x_test = X_train
#y_test = y_train
#model.fit(X_train, y_train, batch_size=128,epochs=7,verbose=1,validation_data=(x_test, y_test))

# save the model to reuse or download
model.save('model.h5')

# NOTE: by default Keras train for 10 epochs
# you will notice that before the 7th-epoch, the validation loss was decreasing then after it increased, so to prevent overfittng, we can nb_epoch = 7

print ('end')


Sample of labels  -0.107229
(160, 320, 3)  shape of train samples
size of training set 8036


Using TensorFlow backend.


Train on 6428 samples, validate on 1608 samples
Epoch 1/7
6428/6428 [==============================] - 4s 575us/step - loss: 8010707.8604 - val_loss: 3837.6371
Epoch 2/7
6428/6428 [==============================] - 4s 575us/step - loss: 3506.7840 - val_loss: 2804.7228
Epoch 3/7
6428/6428 [==============================] - 4s 549us/step - loss: 2487.7966 - val_loss: 1635.2623
Epoch 4/7
6428/6428 [==============================] - 4s 549us/step - loss: 2423.7128 - val_loss: 1407.1920
Epoch 5/7
6428/6428 [==============================] - 4s 555us/step - loss: 2445.2451 - val_loss: 1501.5748
Epoch 6/7
6428/6428 [==============================] - 4s 563us/step - loss: 1835.6224 - val_loss: 1659.7562
Epoch 7/7
6428/6428 [==============================] - 4s 570us/step - loss: 1980.5402 - val_loss: 1690.2780
end


In [ ]:
# Load pickled data
import pickle
import numpy as np
import tensorflow as tf
tf.python.control_flow_ops = tf

with open('./small_traffic_set/small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)

X_train, y_train = data['features'], data['labels']

# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D

# Build Convolutional Neural Network in Keras Here
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

# Preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 )

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, nb_epoch=3, validation_split=0.2)

In [ ]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])